In [ ]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import os

# （可选）Binance API Key（可以为空）
api_key = ''
api_secret = ''

client = Client(api_key, api_secret)

# === 配置参数 ===
symbol = 'DOGEUSDT'  # Dogecoin 的交易对
interval = Client.KLINE_INTERVAL_1HOUR

# 设置开始和结束时间：从 XXXX-XX-XX 开始，抓取 960 小时（40 天）
start_time = datetime(2025, 3, 24)
end_time = start_time + timedelta(hours=960)
start_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')

# 获取 K 线数据（最多 1000 条，抓 960 就够）
klines = client.get_historical_klines(
    symbol=symbol,
    interval=interval,
    start_str=start_str,
    end_str=end_str,
    limit=960
)

# 转换成 DataFrame
df = pd.DataFrame(klines, columns=[
    "timestamp", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base", "taker_buy_quote", "ignore"
])

# 处理时间戳 & 保留主列
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
df = df[["timestamp", "open", "high", "low", "close", "volume"]]
df.set_index("timestamp", inplace=True)

# 获取最后一条记录的日期（UTC）
last_date = df.index[-1].strftime("%Y%m%d")

# 创建保存目录
save_dir = "Dogecoin_Price"
os.makedirs(save_dir, exist_ok=True)

# 文件路径和文件名
file_name = f"Dogecoin_hourly_ohlcv_until_{last_date}.csv"
file_path = os.path.join(save_dir, file_name)

# 保存为 CSV 文件
df.to_csv(file_path)
print(f"✅ 数据保存至: {file_path}")
print(df.tail())


✅ 数据保存至: Dogecoin_Price/Dogecoin_hourly_ohlcv_until_20250502.csv
                           open        high         low       close  \
timestamp                                                             
2025-05-02 19:00:00  0.18081000  0.18081000  0.17958000  0.18004000   
2025-05-02 20:00:00  0.18004000  0.18139000  0.17982000  0.18138000   
2025-05-02 21:00:00  0.18138000  0.18242000  0.18056000  0.18091000   
2025-05-02 22:00:00  0.18091000  0.18146000  0.18058000  0.18097000   
2025-05-02 23:00:00  0.18098000  0.18156000  0.18081000  0.18151000   

                                volume  
timestamp                               
2025-05-02 19:00:00  19832544.00000000  
2025-05-02 20:00:00  10205988.00000000  
2025-05-02 21:00:00  16349889.00000000  
2025-05-02 22:00:00  14023389.00000000  
2025-05-02 23:00:00  10773263.00000000  
